<!--NAVIGATION-->
< [Data Subset: Generic Space-Time Cut](SpaceTime.ipynb) | [Index](Index.ipynb) | [Data Subset: Depth Profile](DepthProfile.ipynb)>

<a href="https://colab.research.google.com/github/simonscmap/pycmap/blob/master/docs/TimeSeries.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<a href="https://mybinder.org/v2/gh/simonscmap/pycmap/master?filepath=docs%2FTimeSeries.ipynb"><img align="right" src="https://mybinder.org/badge_logo.svg" alt="Open in Colab" title="Open and Execute in Binder"></a>

## *time_series(table, variable, dt1, dt2, lat1, lat2, lon1, lon2, depth1, depth2, interval=None)*

Returns a subset of data according to the specified space-time constraints (dt1, dt2, lat1, lat2, lon1, lon2, depth1, depth2). The returned data subset is aggregated by time: at each time interval the mean and standard deviation of the variable values within the space-time constraints are computed. The sequence of these values construct the timeseries. The timeseries data can be binned weekly, monthly, quarterly, or annually, if the **interval** parameter is set (this feature is not applicable to climatological datasets). The resulted timeseries is returned in form of a Pandas datframe ordered by time.

> **Parameters:** 
>> **table: string**
>>  <br />Table name (each dataset is stored in a table). A full list of table names can be found in [catalog](Catalog.ipynb).
>> <br />
>> <br />**variable: string**
>>  <br />Variable short name which directly corresponds to a field name in the table. A subset of this variable is returned by this method according to the spatio-temporal cut parameters (below). A full list of variable short names can be found in [catalog](Catalog.ipynb).
>> <br />
>> <br />**dt1: string**
>>  <br />Start date or datetime. This parameter sets the lower bound of the temporal cut. <br />Example values: '2016-05-25' or '2017-12-10 17:25:00'
>> <br />
>> <br />**dt2: string**
>>  <br />End date or datetime. This parameter sets the upper bound of the temporal cut. 
>> <br />
>> <br />**lat1: float**
>>  <br />Start latitude [degree N]. This parameter sets the lower bound of the meridional cut. Note latitude ranges from -90&deg; to 90&deg;.
>> <br />
>> <br />**lat2: float**
>>  <br />End latitude [degree N]. This parameter sets the upper bound of the meridional cut. Note latitude ranges from -90&deg; to 90&deg;.
>> <br />
>> <br />**lon1: float**
>>  <br />Start longitude [degree E]. This parameter sets the lower bound of the zonal cut. Note latitude ranges from -180&deg; to 180&deg;.
>> <br />
>> <br />**lon2: float**
>>  <br />End longitude [degree E]. This parameter sets the upper bound of the zonal cut. Note latitude ranges from -180&deg; to 180&deg;.
>> <br />
>> <br />**depth1: float**
>>  <br />Start depth [m]. This parameter sets the lower bound of the vertical cut. Note depth is a positive number (it is 0 at surface and grows towards ocean floor).
>> <br />
>> <br />**depth2: float**
>>  <br />End depth [m]. This parameter sets the upper bound of the vertical cut. Note depth is a positive number (it is 0 at surface and grows towards ocean floor).
>> <br />
>> <br />**interval: None or string, default: None**
>>  <br />The timeseries bin size. If None, the native dataset time resolution is used as the bin size. <br />Below is a list of interval values for other binning options:
* **'w'** or **'week'** for weekly timeseries.
* **'m'** or **'month'** for monthly timeseries.
* **'s'** or **'q'** for seasonal/quarterly timeseries.
* **'a'** or **'y'** for annual/yearly timeseries.

>**Returns:** 
>>  Pandas dataframe.


### Example 1

This example retrieves the timeseries of SiO4 measurements conducted by [HOT team](http://hahana.soest.hawaii.edu/hot/hot-dogs/) at University of Hawaii, spanning from 1988 to 2016. 

In [ ]:
#!pip install pycmap -q     #uncomment to install pycmap, if necessary

import pycmap

api = pycmap.API(token='<YOUR_API_KEY>')
api.time_series(
               table='tblHOT_Bottle', 
               variable='SiO4_bottle_hot', 
               dt1='1988-12-01', 
               dt2='2016-10-15', 
               lat1=22, 
               lat2=23, 
               lon1=-159, 
               lon2=-157, 
               depth1=0, 
               depth2=0
               ) 

### Example 2

This example retrieves a 24-year long timeseries of absolute dynamic topography (closely related to sea surface height) measured by satellite. 
<br />Notice, depth1 and depth2 values are automatically ignored because this is a surface dataset. The 'interval' parameter (line 24) has to 'y' indicating yearly binning (inter-annual timeseres). This example takes a few moments to run as the altimetry dataset is very large (multi-decade daily-global remote sensing). 
<br />The last few lines of code (lines 28-32) makes a simple plot to visualize the retrieved data.

In [ ]:
#!pip install pycmap -q     #uncomment to install pycmap, if necessary

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pycmap


   

api = pycmap.API(token='<YOUR_API_KEY>')
table, variable = 'tblAltimetry_REP', 'adt'
df = api.time_series(
                   table=table, 
                   variable=variable, 
                   dt1='1994-01-01', 
                   dt2='2017-12-31', 
                   lat1=30, 
                   lat2=32, 
                   lon1=-160, 
                   lon2=-158, 
                   depth1=0, 
                   depth2=0,
                   interval='y'
                   ) 


plt.errorbar(df['year'], df['adt'], yerr=df['adt_std'], fmt='ob', capsize=3, alpha=0.4)
plt.fill_between(df['year'], df['adt']-df['adt_std'], df['adt']+df['adt_std'], color='gray', alpha=0.2)
plt.xlabel('Year')
plt.ylabel(api.get_var_long_name(table, variable) + api.get_unit(table, variable))
plt.show()

<img src="figures/sql.png" alt="SQL" align="left" width="40"/> 
<br/>
### SQL Statement
Here is how to achieve the same results using a direct SQL statement. Please refere to [Query](Query.ipynb) for more information.

<code>EXEC uspTimeSeries 'tableName', 'variable', 'dt1', 'dt2', 'lat1', 'lat2', 'lon1', 'lon2', 'depth1', 'depth2'</code>


**Examples (different intervals):**<br/>
<code>EXEC uspTimeSeries 'tblsst_AVHRR_OI_NRT', 'sst', '2016-01-01', '2016-12-31', '20', '25', '-160', '-158', '0', '0'</code>

<code>EXEC uspWeekly 'tblsst_AVHRR_OI_NRT', 'sst', '2016-01-01', '2016-12-31', '20', '25', '-160', '-158', '0', '0'</code>

<code>EXEC uspMonthly 'tblsst_AVHRR_OI_NRT', 'sst', '2016-01-01', '2016-12-31', '20', '25', '-160', '-158', '0', '0'</code>

<code>EXEC uspQuarterly 'tblsst_AVHRR_OI_NRT', 'sst', '2016-01-01', '2016-12-31', '20', '25', '-160', '-158', '0', '0'</code>

<code>EXEC uspAnnual 'tblsst_AVHRR_OI_NRT', 'sst', '2015-01-01', '2018-12-31', '20', '25', '-160', '-158', '0', '0'</code>

